## Exploratory Analysis for Police Summary Reports

### Packages

In [3]:
# pip install transformers

In [4]:
import os
import re
import random
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

c:\Users\fdmol\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\fdmol\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\fdmol\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<ipython-input-4-33c173740434>:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
     

### Parameters

In [5]:
PATH = "C:/Users/fdmol/Desktop/MSCAPP/CAPP30255/NLP-Police-Complaints/data/text_files"
CHARS_TO_REMOVE = ["\n", "§"]
REGEX_PATTERNS = [
    r"civilian office of police accountability\s+",
    r"log\s*\#\s*\d+",
    r"-\s*\d+\s*\d+",
    r"summary report of investigation\s+",
    r"i.\s+executive\s+summary",
    r"_+",
    r"\s*date of incident:\s*\w+\s+\d+,\s+\d+",
    r"\s*time of incident:\s*\d+:\d+\w+",
    r"\s*location of incident:\s*\d+\w+\s*\w+",
    r"\s*date of copa notification:\s*\w+\s+\d+,\s+\d+",
    r"\s*time of copa notification:\s*\d+:\d+\w+",
    r"applicable rules and laws|"
    r"conclusion|"
    r"digital evidence|"
    r"documentary evidence|"
    r"legal standard|",
    r"appendix\s+.*",
    r"\s+deputy chief administrator\s+",
    r"\s+deputy chief investigator\s+",
    r"\s+ibid\s+",
]

#### Functions

I define a class for reading and processing the data, I took some ideas from Matt's analysis to remove headers and other elements that are not relevant to us. This could also help in getting better results for the summarization task.

In [6]:
class TextParser:
    CHARS_TO_REMOVE = CHARS_TO_REMOVE
    REGEX_PATTERNS = REGEX_PATTERNS

    def __init__(self, path):
        self.path = path

    def txt_to_list(self, filename):
        """
        Add each line of a text file to a list
        """

        file_path = os.path.join(self.path, filename)
        lines = []
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip().split()
                lines.append(line)

        return lines

    def file_to_string(self, filename):
        """
        Add each line of a text file to a string
        """
        text = ""
        file_path = os.path.join(self.path, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                for char in self.CHARS_TO_REMOVE:
                    line = line.replace(char, "")
                text += line

        text = text.strip()
        text = text.lower()
        text = re.sub(r"\s+", " ", text)

        # Remove REGEX patterns
        for pattern in self.REGEX_PATTERNS:
            text = re.sub(pattern, "", text, flags=re.IGNORECASE)
        return text

The function below wraps HuggingFace's tokenizer and model to generate a summary for each complaint. As I mention below, I tweaked the parameters to get better summaries.

In [11]:
def generate_summary(complaint_text, tokenizer, model):
    """
    Generates a summary of a complaint given
    the complaint text
    """
    # Tokenize the text
    inputs = tokenizer(
        complaint_text, return_tensors="pt", max_length=2512, truncation=True
    )

    # Generate summary
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=1200,
        min_length=40,
        length_penalty=2.0,
        no_repeat_ngram_size=2,
        num_beams=4,
        early_stopping=True,
    )

    # Decode and print the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

In [12]:
text_parser = TextParser(PATH)
complaints = os.listdir(PATH)
complaints = [complaint for complaint in complaints if complaint.endswith(".txt")]
complaints = random.sample(complaints, 10)


for complaint in complaints:
    complaint_text = text_parser.file_to_string(complaint)
    print(complaint)
    print("=====================================")
    # print(complaint_text)


# complaint_text

2012-1052278.txt
1089507.txt
2019-0004415.txt
2015-1075421.txt
2008-1017636.txt
2017-1087307.txt
1074294.txt
2019-0003101.txt
2021-0002951.txt
2014-1069136.txt


#### NLP Task: Summarization

I will use this model:

https://huggingface.co/docs/transformers/main/en/model_doc/t5#transformers.T5ForConditionalGeneration

In the cells below, I use the model to generate a summary for ten random complaints. The summaries are of medium quality, depending on each complaint. 

I did the following to try to improve the quality of the summaries:

- Adjuster the `max_length` parameter to limit the length of the summary
- Adjusted the `min_length` parameter to ensure the summary is at least a certain length
- Adjusted the `num_beams` parameter to increase the number of beams used in beam search
- Adjusted `no_repeat_ngram_size` parameter to avoid repeating n-grams in the summary

I also experimented with the max_length of the tokens used in the tokenizer.

In [13]:
model_name = "Falconsai/text_summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [15]:
text_parser = TextParser(PATH)

# Get a random list of 10 complaints
complaints = os.listdir(PATH)
complaints = [complaint for complaint in complaints if complaint.endswith(".txt")]
complaints = random.sample(complaints, 3)


for complaint in complaints:
    complaint_text = text_parser.file_to_string(complaint)
    summary = generate_summary(complaint_text, tokenizer, model)
    print(f"Complaint: {complaint}")
    print("=====================================")
    print(f"Summary: {summary}\n\n")

Complaint: 2021-0001489.txt
Summary: officer jason bala is alleged to have been detained without justification. cpd officers entered and searched the residence at apt #2, the civilian office of poli ce accountability summary report of investigation1 date /time/location of incident: january 16, 2020 at approximately 3:53 pm, at or chicago, il, accused committed misconduct through the following acts or omissions: 1. a complaint on behalf of the complainant, via the bureau of internal affairs (“bia”)4.


Complaint: 2016-1080339.txt
Summary: (witness): involved individual #1 (wits), #6 (sergeant, employee #, appointed, 1995; white male involved officer #). officer yanked his arm away and pulled away from him and announced that she was a police officer. iv. rules rule 1: insubordination of disrespect toward supervisory member on or off duty. rule 2: any action or conduct which impedes the department’s efforts to achieve its policy and goals.


Complaint: 2020-0000630.txt
Summary: officer ra

Some next steps could be:

- Improve cleaning process to remove irrelevant headers and footers
- Try other models
- Finetune a model on this dataset (we would need to create a labeled dataset for this, and possibly generate the summaries by hand).